# LSTM Network

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam



In [2]:
df=pd.read_csv('Bruska_data.csv')
print(df.head())
print(df.isnull().sum())

            time stamp  Bruska_wind speed  Bruska_wind direction  \
0  2018-01-01 00:00:00           9.363334              172.79999   
1  2018-01-01 01:00:00           9.266666              174.99990   
2  2018-01-01 02:00:00           9.586667              174.19984   
3  2018-01-01 03:00:00           9.320000              167.53835   
4  2018-01-01 04:00:00           9.267742              160.91630   

   Bruska_air density  Zelengrad_wind speed  Zelengrad_wind direction  \
0               1.210             10.635238                 165.28897   
1               1.210             10.261664                 166.09566   
2               1.209             13.286666                 161.92586   
3               1.208             14.777143                 159.17044   
4               1.207             11.447466                 151.64941   

   Zelengrad_air density  Bruska_active power output  \
0                  1.207                   21.608133   
1                  1.207                

In [3]:
df.dropna(inplace=True)

In [4]:
df.drop(["time stamp"],axis=1,inplace=True)

In [5]:
df.drop(["Bruska_normally operating number"],axis=1,inplace=True)

In [6]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['Bruska_active power output']])

In [7]:
X_train1, X_test1, Y_train, Ytest = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)

Train set shape (13751, 7)
testing set shape (3438, 7)


In [8]:
X_train1=np.reshape(X_train1,(13751,7,1))
X_test1=np.reshape(X_test1,(3438,7,1))
print('Train set shape', X_train1.shape)
print('Testing set shape', X_test1.shape)

Train set shape (13751, 7, 1)
Testing set shape (3438, 7, 1)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=32, return_sequences=True,input_shape=(X_train1.shape[1], X_train1.shape[2])),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=1, activation='linear', name='output_layer')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_absolute_error')

In [10]:
import time
start= time.time()
history = model.fit(X_train1, Y_train, epochs=100, 
                    verbose=1, batch_size=32)
end= time.time()
convergence_time= end-start
print("convergence time:",convergence_time)

Epoch 1/100
430/430 [==============================] - 5s 6ms/step - loss: 0.0601
Epoch 2/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0327
Epoch 3/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0306
Epoch 4/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0294
Epoch 5/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0296
Epoch 6/100
430/430 [==============================] - 3s 6ms/step - loss: 0.0296
Epoch 7/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0285
Epoch 8/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0285
Epoch 9/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0281
Epoch 10/100
430/430 [==============================] - 3s 6ms/step - loss: 0.0278
Epoch 11/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0273
Epoch 12/100
430/430 [==============================] - 2s 6ms/step - loss: 0.0278
Epoch 13/100


In [11]:
import time
start= time.time()
prediction_lstm=model.predict(X_test1)
end= time.time()
continential= end -start
print("inference time:",continential)

108/108 [==============================] - 1s 3ms/step
inference time: 1.0563054084777832


In [12]:
prediction_lstm
print(prediction_lstm.shape)
pred=prediction_lstm.flatten()
print(pred.shape)
prediction = pred.reshape(pred.shape[0], 1)
prediction= s2.inverse_transform(prediction)
Actual2= s2.inverse_transform(Ytest)
x= prediction
z= Actual2
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE

max = x.max()
min = x.min()
mean = x.mean()
print("Maximum:",max)
print('Minimum:',min)
print("Mean:",mean)
NRMSE= (RMSE/(max-min))*100
print('Normalized Root Mean Square Error is:',NRMSE)
print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)


Calculated = pd.DataFrame(x, columns = ['Prediction'])
Actual = pd.DataFrame(z, columns = ['Actual'])
combined_df = pd.concat([Calculated, Actual], axis=1)




(3438, 1)
(3438,)
Maximum: 36.855534
Minimum: -0.65797234
Mean: 12.138851
Normalized Root Mean Square Error is: 1.2773736155769029
Root Mean Square Error: 0.4791876027361423
Mean Square Error: 0.22962075861601094
Mean Absolute Error: 0.37346299170095665


In [13]:
combined_df.to_csv(r'C:\Users\sunrise\Desktop\Wind speed forecasting\Bruska\LSTM\LSTM_output.csv', index=False)